Build a regression model.

 Histograms and heatmap provided on joining data file

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

Provide model output and an interpretation of the results. 

In [2]:
yelps_df=pd.read_csv("yelps_table.csv")

In [3]:
fs_df=pd.read_csv("fs_table.csv")

In [4]:
bike_df=pd.read_csv("citybikestable.csv")

In [5]:
## foursquare dataframe
fs_df

,name,distance,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location,location.address
0,Rogue Kitchen & Wetbar,250,49.263262,-123.117650,0,602W Broadway W
1,33 Acres Brewing Co,674,NaN,NaN,0,15 8th Ave W
2,The Cascade Room,984,NaN,NaN,0,2616 Main St
3,Biercraft Bistro,744,NaN,NaN,0,3305 Cambie St
4,Faculty Brewing Co,994,49.268561,-123.104665,0,1830 Ontario St
5,Fox Cabaret,965,NaN,NaN,0,2321 Main St
6,The Wicklow Pub,760,NaN,NaN,0,610 Stamp's Landng
7,Tocador,996,NaN,NaN,0,2610 Main St
8,Vancouver Hackspace,820,NaN,NaN,0,1715 Cook St
9,Mahony & Sons,811,NaN,NaN,0,610 Stamp's Landng


In [6]:
## yelps dataframe
yelps_df

,name,rating,latitude,longitude
0,Bike Repair Centre,5.0,49.263173,-123.105946
1,Our Community Bikes,4.0,49.263217,-123.101212
2,Mobi by Shaw Go,1.5,49.260380,-123.113358


In [7]:
## bikes dataframe
bike_df

,latitude,longitude,bikes
0,49.262487,-123.114397,21
1,49.274566,-123.121817,4
2,49.279764,-123.110154,5
3,49.260599,-123.113504,3
4,49.264215,-123.117772,15
...,...,...,...
239,49.247204,-123.101549,1
240,49.285670,-123.112543,6
241,49.250940,-123.101306,1
242,49.280977,-123.035969,4


In [8]:
total_df=fs_df[["name","distance"]]

In [9]:
total_df["rating"]=0

C:\Users\Alex\AppData\Local\Temp\ipykernel_10076\1805191628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df["rating"]=0


In [10]:
total_df=pd.concat([total_df,yelps_df])

In [11]:
total_df.replace(0,np.nan,inplace=True)

In [12]:
total_df["bikes"]=bike_df.loc[0,"bikes"]

In [13]:
##combined dataframe
total_df

,name,distance,rating,latitude,longitude,bikes
0,Rogue Kitchen & Wetbar,250.0,NaN,NaN,NaN,21
1,33 Acres Brewing Co,674.0,NaN,NaN,NaN,21
2,The Cascade Room,984.0,NaN,NaN,NaN,21
3,Biercraft Bistro,744.0,NaN,NaN,NaN,21
4,Faculty Brewing Co,994.0,NaN,NaN,NaN,21
5,Fox Cabaret,965.0,NaN,NaN,NaN,21
6,The Wicklow Pub,760.0,NaN,NaN,NaN,21
7,Tocador,996.0,NaN,NaN,NaN,21
8,Vancouver Hackspace,820.0,NaN,NaN,NaN,21
9,Mahony & Sons,811.0,NaN,NaN,NaN,21


In [14]:
bike_df.describe()

,latitude,longitude,bikes
count,244.000000,244.000000,244.000000
mean,49.271950,-123.116262,7.831967
std,0.011780,0.028844,6.006893
min,49.238105,-123.191458,0.000000
25%,49.263484,-123.135466,3.000000
50%,49.271596,-123.116793,6.000000
75%,49.280859,-123.097554,11.000000
max,49.302577,-123.035969,27.000000


In [15]:
##generating the ordinary least squares regression to identify r/ship between bikes and POI's
x=sm.add_constant(bike_df[["latitude","longitude"]])
model=sm.OLS(bike_df["bikes"],x)
results=model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  bikes   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     13.86
Date:                Wed, 23 Aug 2023   Prob (F-statistic):           2.01e-06
Time:                        17:34:19   Log-Likelihood:                -769.91
No. Observations:                 244   AIC:                             1546.
Df Residuals:                     241   BIC:                             1556.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.075e+04   2122.819     -5.065      0.000   -1.49e+04   -6570.976
latitude     139.4869     31.171      4.475      0.000      78.084     200.890
longitude    -31.5772     12.730     -2.480      0.014     -56.654      -6.500
==============================================================================
Omnibus:                       26.521   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.746
Skew:                           0.827   Prob(JB):                     1.28e-07
Kurtosis:                       3.624   Cond. No.                     7.70e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.7e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# from this regression plot, 
-the adjusted R-squared shows model is fit to explain patterns of data with 0.096, the independent variable can only explain 0.1% of the observed dependent variable 
-with the P-value less than the 0.05 threshold means that these are statistically significant variables
- Eventhough these variables are statistically significant, the number of bikes is still dependent on more significant factors.

# Stretch

How can you turn the regression model into a classification model?

Having difficulty actualising this.